In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df_final = pd.read_csv('../../data/final.csv')
df_pred = pd.read_csv('predicted_actives_total.csv')
df_pred.head(5)

,USER_ID,D001,D002,D003,D004,D005,D006,D007,D008,D009,...,D770,D771,D772,D773,D774,D775,D776,D777,Predictions,Probabilities
0,D488-0121,3,0,0,1,4,0,0,0,0,...,0,0,0,0,5.72792,-0.822058,0.461538,3.66371,1,0.594959
1,D488-0114,2,0,0,1,4,0,0,0,0,...,0,0,0,0,5.70044,-0.816815,0.315789,3.33826,1,0.659974
2,D305-0402,3,0,0,1,4,0,0,0,0,...,0,0,0,0,5.61471,-0.816815,0.473684,4.08437,1,0.527001
3,F069-0210,2,0,0,0,3,0,0,0,0,...,0,0,0,0,5.28540,-0.706842,0.400000,2.58441,1,0.521137
4,D420-2730,2,0,0,1,4,0,0,0,0,...,0,0,0,0,5.49185,-0.767101,0.342857,3.00023,1,0.517550


In [2]:
df_final.head()

,D001,D002,D003,D004,D005,D006,D007,D008,D009,D010,...,D769,D770,D771,D772,D773,D774,D775,D776,D777,Activity
0,2,0,0,1,4,0,0,0,0,0,...,0,1,0,0,0,5.45943,-0.367670,0.363636,4.29650,0
1,1,0,0,2,1,0,0,0,0,0,...,0,1,0,0,0,5.08746,-0.772128,0.240000,2.31134,0
2,1,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,4.70044,-0.295978,0.300000,1.67610,0
3,2,0,0,1,3,0,0,0,0,0,...,0,1,1,0,0,5.49185,-0.357977,0.352941,2.63932,0
4,1,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,4.64386,-0.274183,0.315789,1.41996,0


In [3]:
# dropping coloumns that are not needed
df_pred.drop(['USER_ID', 'Probabilities', 'Predictions'], axis=1, inplace=True)

In [4]:
df_pred.head()

,D001,D002,D003,D004,D005,D006,D007,D008,D009,D010,...,D768,D769,D770,D771,D772,D773,D774,D775,D776,D777
0,3,0,0,1,4,0,0,0,0,0,...,0,0,0,0,0,0,5.72792,-0.822058,0.461538,3.66371
1,2,0,0,1,4,0,0,0,0,0,...,0,0,0,0,0,0,5.70044,-0.816815,0.315789,3.33826
2,3,0,0,1,4,0,0,0,0,0,...,0,0,0,0,0,0,5.61471,-0.816815,0.473684,4.08437
3,2,0,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,5.28540,-0.706842,0.400000,2.58441
4,2,0,0,1,4,0,0,0,0,0,...,0,0,0,0,0,0,5.49185,-0.767101,0.342857,3.00023


In [6]:
#Removing low variance columns
from sklearn.feature_selection import VarianceThreshold
vt = VarianceThreshold(threshold=(.8*(1-.8)))

#Split dataset into training set and testing set
X = df_final.drop(['Activity'],axis=1)
y = df_final['Activity']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Fit
_ = vt.fit(X_train)

#Get the mask
mask = vt.get_support()

#Subset the DataFrame
X_train = X_train.loc[:, mask]
X_test = X_test.loc[:,mask] # not needed since we are not using it

#Subset pred dataset
df_pred = df_pred.loc[:, mask]

#Renaming X_train as df_train
df_train = X_train

#Checking the shape of the datasets
df_train.shape, df_pred.shape

((156195, 359), (45, 359))